In [3]:
import pandas as pd#импорт библиотеки pandas
data = pd.read_csv('/datasets/data.csv')

In [4]:
data.info()#получение информации о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Типы данных во всех столбцах разные - float, int, object.


**Согласно документации к данным** :

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита.


Количество значений в столбцах различается,значит,есть пропущенные значения.

**Вывод**

Данные в таблице описывают клиента. Можно подумать,что данных достаточно для проверки гипотез. Но встречаются пропущенные значения,отрицательные числа, а также ошибки в распознавании типов данных.
Чтобы двигаться дальше, нужно устранить проблемы в данных.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
data.isna().sum()#количество пропущенных значений

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения влияют на ход работы,поэтому выведем пропуски,чтобы понять,к какому типу они принадлежат. Затем заполним их.

In [6]:
data[data['days_employed'].isna()].head()#вывод пропущенных значений по столбцу days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Мы видим,что пропуски оозначены NaN- они принадлежат к типу float. Можно сделать вывод, что пропущенные значения относятся к количественным переменным. Также,судя из таблицы,можно сказать,что пропуски в столбцах days_employed, total_income совпадают в одних и тех же строках.

Приступим к заполнению пропусков с помощью loc и медианы.

In [7]:
#заполнение пропусков в столбце days_employed
data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'].median()
#заполнение пропусков в столбце total_income
data.loc[data['total_income'].isna(), 'total_income'] = data['total_income'].median()

Убедимся,что больше нет пропусков. Для этого еще раз проверим количество пропущенных значений.

In [8]:
data.isna().sum()#количество пропущенных значений

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Я заметила аномальные числа в стобце days_employed. Скорее всего некоторые значения в столбце заполнены не в днях,а в часах.
Переведем в дни и запишем новые данные в столбец days_employed_1.

In [9]:
data.loc[data['days_employed'] > 10000, 'days_employed_1'] = data.loc[data['days_employed'] > 10000, 'days_employed'] / 24#перевод в дни
data[['days_employed', 'days_employed_1']]

,days_employed,days_employed_1
0,-8437.673028,NaN
1,-4024.803754,NaN
2,-5623.422610,NaN
3,-4124.747207,NaN
4,340266.072047,14177.753002
...,...,...
21520,-4529.316663,NaN
21521,343937.404131,14330.725172
21522,-2113.346888,NaN
21523,-3112.481705,NaN


Мы завершили работу над заполнением пропусков.

### Замена типа данных

**Для дальнейшей работы нам требуется**:
- заменить тип данных в столбцах days_employed, total_income на int с помощью метода astype(), т.к это универсальный метод,в аргумент которого передается нужный тип
- перевести отрицательные числа в положительные с помощью .apply(abs)

In [10]:
#заменяем тип данных на int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
#отрицательные числа заменяем на положительные
data['days_employed'] = data['days_employed'].apply(abs)
data['total_income'] = data['total_income'].apply(abs)

### Обработка дубликатов

In [11]:
data.duplicated().sum()#подсчет дубликатов

54

In [12]:
data = data.drop_duplicates().reset_index(drop=True)#удаление дубликатов с восстановлением индексов

In [13]:
data.duplicated().sum()#проверка на отсутствие дубликатов

0

Приведем значения в столбце education к нижнему регистру:

In [14]:
data['education'] = data['education'].str.lower()

**Вывод**

Мы удалили явные дубликаты и привели столбец education к нижнему регистру. Удалять дубликаты в этом столбце не требуется.

### Лемматизация

Найдем уникальные значения в столбце purpose:

In [15]:
data['purpose'].unique()#уникальные значения в столбце purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [16]:
from pymystem3 import Mystem#импортируем pymystem
m = Mystem()
lemmas = m.lemmatize(''.join(data['purpose']))#склеиваем столбец в строку
from collections import Counter
print(Counter(lemmas))#подсчитываем число упоминаний слов в тексте

Counter({' ': 33596, 'с': 2918, 'свой': 1705, 'покупка': 1456, 'коммерческий': 1312, 'жилье': 1290, 'для': 1290, 'жилой': 1231, 'высокий': 922, 'подержать': 896, 'недвижимостипокупка': 799, 'проведение': 773, 'собственный': 635, 'со': 627, 'недвижимостиоперация': 462, 'дополнительный': 447, 'автомобиляпокупка': 414, 'недвижимостина': 396, 'недвижимостьюпокупка': 390, 'недвижимостистроительство': 311, 'образованиепокупка': 271, 'образованияпокупка': 263, 'жильяпокупка': 263, 'недвижимостьюоперация': 253, 'автомобиляоперация': 222, 'недвижимостиполучение': 219, 'образованиемпокупка': 210, 'недвижимостьюн': 198, 'автомобилястроительство': 187, 'автомобилемпокупка': 185, 'автомобиляна': 183, 'свадьбупокупка': 173, 'недвижимостизаняться': 171, 'свадьбыпокупка': 167, 'образованиеоперация': 164, 'недвижимостьюстроительство': 164, 'недвижимостисделка': 161, 'образованияоперация': 157, 'сдачипокупка': 154, 'жильяоперация': 151, 'жильяна': 140, 'жильемпокупка': 131, 'образованиян': 131, 'жильюпо

Проанализировав полученный результат, мы видим, что многие уникальные значения цели кредита названы похожим образом, но имеют одинаковый смысл. Мы лемматизируем их, чтобы затем разбить на категории. Мы выбираем в ручную наиболее часто встречающиеся слова и с и apply() создаем и заполняем значениями функции новый столбец с категориями целей.

In [17]:
def purpose_category(row):
    categories = ['автомобиль', 'жилье', 'недвижимость', 'образование', 'свадьба', 'строительство']
    lemmas = m.lemmatize(row) # тут нужно лемматизировать row т.е. это значение текущей ячейки
    for word in categories:
        if word in lemmas:
           return word
 
data['purpose_category'] = data['purpose'].apply(purpose_category) 

Подсчитаем частоту значений в этом столбце:

In [18]:
data['purpose_category'].value_counts()#частота значений

недвижимость    6353
жилье           4461
автомобиль      4308
образование     4014
свадьба         2335
Name: purpose_category, dtype: int64

### Категоризация данных

Выделим словарь уникальных значений для семейного положения:

In [19]:
data_family_status = data[['family_status']].drop_duplicates().reset_index(drop=True)
display(data_family_status)

,family_status
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


Выделим словарь уникальных значений для кол-ва детей:

In [20]:
data_children = data[['children']].drop_duplicates().reset_index(drop=True)
display(data_children)

,children
0,1
1,0
2,3
3,2
4,-1
5,4
6,20
7,5


Мы видим некорректные значения в столбце с кол-вом детей. -1 - отрицательное число,а также 20(скорее всего произошла ошибка при заполнении). Так как мы не знаем,как произошла ошибка, 20 заменим медианным значением.

In [21]:
data['children'] = data['children'].replace(-1, 1)
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_median)
data_children = data[['children']].drop_duplicates().reset_index(drop=True)
display(data_children)

,children
0,1.0
1,0.0
2,3.0
3,2.0
4,4.0
5,5.0


Категоризируем данные столбца total_income. Для этого напишем функцию,которая будет проверять сумму дохода и возвращать категорию уровня дохода в новый столбец total_income_group с помощью метода apply()

In [22]:
total_income_group = data[['debt', 'total_income']]
total_income_group = total_income_group.drop_duplicates().reset_index(drop=True)
def total_income_group(total_income):
    if total_income < 80000:
        return 'низкий доход'
    if 80000 <= total_income < 200000:
        return 'средний доход'
    return 'высокий доход'
data['total_income_group'] = data['total_income'].apply(total_income_group)

**Вывод**

Мы категоризировали все нужные нам столбцы. Теперь с их помощью мы можем двигаться дальше и проверять гипотезы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

С помощью pivot_table формируем сводную таблицу, где отображаем кол-во должников, кол-во долгов, процент невозврата.

In [23]:
#формируем сводную таблицу
debt_from_children = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count', 'mean']).fillna(0)
debt_from_children.columns = ['debt', 'total', 'percent']
#сортируем в порядке убывания по проценту невозврата
debt_from_children.sort_values(by='percent', ascending=False)

,debt,total,percent
children,,,
4.0,4,41,0.097561
2.0,194,2052,0.094542
1.0,445,4856,0.091639
3.0,27,330,0.081818
0.0,1071,14183,0.075513
5.0,0,9,0.000000


**Вывод**

С увеличением количества детей мы действительно видим увеличение задолженностей. Люди с 3 детьми платят в срок чаще,чем люди с другим кол-вом детей. Также, можно отметить,что бездетные реже просрачивают оплату по кредиту. По данным,которые мы имеем,можно сказать,что зависимость есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Также с помощью pivot_table повторяем действия

In [24]:
debt_from_family_status = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count', 'mean']).fillna(0)
debt_from_family_status.columns = ['debt', 'total', 'percent']
debt_from_family_status.sort_values(by='percent', ascending=False)

,debt,total,percent
family_status,,,
Не женат / не замужем,274,2810,0.097509
гражданский брак,388,4163,0.093202
женат / замужем,931,12344,0.075421
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


**Вывод**

Да,зависимость есть. Люди,никогда не бывавшие в браке чаще не платят в срок. Но люди в разводе или овдовевшие чаще замужних платят вовремя.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

С помощью pivot_table совершаем аналогичные действия:

In [25]:
debt_from_total_income_group = data.pivot_table(index = ['total_income_group'], values = 'debt', aggfunc = ['sum', 'count', 'mean']).fillna(0)
debt_from_total_income_group.columns = ['debt', 'total', 'percent']
debt_from_total_income_group.sort_values(by='percent', ascending=False)

,debt,total,percent
total_income_group,,,
средний доход,1209,14129,0.085569
низкий доход,174,2276,0.076450
высокий доход,358,5066,0.070667


**Вывод**

Мы видим,что зависимости практически нет,т.к различия несущественны. Но люди с низким и высоким уровнем дохода все таки чаще платят кредит в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [26]:
debt_from_purpose_category = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ['sum', 'count', 'mean']).fillna(0)
debt_from_purpose_category.columns = ['debt', 'total', 'percent']
debt_from_purpose_category.sort_values(by='percent', ascending=False)

,debt,total,percent
purpose_category,,,
автомобиль,403,4308,0.093547
образование,370,4014,0.092177
свадьба,186,2335,0.079657
недвижимость,474,6353,0.074610
жилье,308,4461,0.069043


**Вывод**

"Недвижимость", "жилье" и "свадьба" просрачивают реже. А вот в категориях "автомобиль" и "образование" большой процент невозврата.

## Шаг 4. Общий вывод

Мы проверили 4 гипотезы и установили : 


1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Первая гипотеза подтвердилась,действительно наличие детей влияет на возврат кредита в срок.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?


Вторая гипотеза также подтвердилась.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


С уровнем дохода все сложнее,можно сказать,что гипотеза подтвердилась лишь отчасти,так как отклонения незначительны.

4. Как разные цели кредита влияют на его возврат в срок.


Мы установили ,что в категориях "автомобиль" и "образование" самый большой процент невозврата по сравнению с остальными.